In [2]:
from data import dataloader
from GPT import GPT
from nltk.tokenize import WordPunctTokenizer
import torch.nn as nn
import torch
import config

In [3]:
from word_sequence import Word2Sequence
from nltk.tokenize import WordPunctTokenizer
tokenizer = Word2Sequence()
tokenizer.load_dict(config.w2s_dict_path)

Successfully Loaded label dict!


In [4]:
gpt = GPT(
    vocab_size = config.vocab_size,
    max_len = config.max_len,
    d_model = config.d_model,
    nhead = config.nheads,
    dim_feedforward = config.dim_feedforward,
    num_layers = config.decoder_layers
)
gpt.to(config.device)
gpt.load_state_dict(torch.load("GPT.pt"))

<All keys matched successfully>

In [5]:
import random
def select_top_k(predictions, current_loc, k=1):
    predicted_index = random.choice(
        predictions[0, current_loc-1, :].sort(descending=True)[1][:k]).item()
    return predicted_index

def get_curr_loc(target):
    idx = 0
    for i in target:
        if i != 0:
            idx += 1
        else:
            return idx

In [6]:
def generate(model, x, k=1):
    target = ['<SOS>'] + WordPunctTokenizer().tokenize(x.lower())
    target = tokenizer.transform(target, max_len=128, pad_first=False)
    loc = get_curr_loc(target)
    target = torch.LongTensor(target).unsqueeze(0)
    
    for i in range(config.sent_length-loc-1):
        target = target.to(config.device)
        out = model(target)
        pred = select_top_k(out, loc, k=k)
        if pred == 2:
            i -= 1
            continue
        target[0][loc] = pred
        loc += 1
    return target

In [12]:
x = "POLIXENES:"
p = generate(gpt, x, k=1)
print(" ".join(tokenizer.inverse_transform(p.cpu()[0], is_tensor=True)))

<SOS> polixenes : we know ' t , we know ' t . first citizen : let us kill him , and we ' ll have corn at our own price . is ' t a verdict ? all : no more talking on ' t ; let it be done : away , away ! second citizen : one word , good citizens . first citizen : we are accounted poor citizens , the patricians good . what authority surfeits on would relieve us : if they would yield us but the superfluity , while it were wholesome , we might guess they relieved us humanely ; but they think we are too dear : the leanness that afflicts us , the object of our misery <PAD>


In [ ]:
# Seems like the model successfully worked... generation pretty bad, try tune up d_model